# Discounted Cash Flow Valuation of BMW AG: A Quantitative Framework

**Author:** Financial Analysis Research Team  
**Date:** November 2025  
**Subject:** BMW.DE (XETRA)  
**Classification:** Consumer Cyclical — Auto Manufacturers

---

## Abstract

This technical document presents a systematic approach to equity valuation using Discounted Cash Flow (DCF) methodology applied to Bayerische Motoren Werke AG. The analysis integrates historical financial statement data, peer group benchmarking, and capital efficiency metrics to derive an estimate of intrinsic value. Our methodology emphasizes transparency and reproducibility through computational implementation in Python.

**Keywords:** DCF Valuation, Free Cash Flow, ROIC, Automotive Industry, Equity Analysis

## 1. Introduction

### 1.1 Research Objective

The primary objective of this analysis is to estimate the intrinsic value of BMW AG common equity using a multi-stage Discounted Cash Flow model. This valuation framework follows established corporate finance principles as documented in Damodaran (2012) and Koller et al. (2020).

### 1.2 Industry Context

The European automotive sector faces a period of structural transformation driven by regulatory requirements for emissions reduction and accelerating consumer adoption of electric vehicles. BMW AG operates within the premium segment, where brand positioning and technological differentiation remain critical competitive factors.

### 1.3 Analytical Framework

This analysis employs a three-stage methodology:

1. **Data Collection**: Extraction of financial statements and market data via standardized API interfaces
2. **Historical Analysis**: Examination of profitability trends, capital allocation, and cash flow generation
3. **Peer Benchmarking**: Comparative assessment against industry participants to contextualize findings

All computational procedures are implemented using documented, version-controlled code to ensure reproducibility.

In [75]:
# =============================================================================
# Environment Configuration
# =============================================================================
# Standard library imports
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Project-specific modules for data retrieval and visualization
%load_ext autoreload
%autoreload 2
from ds import data, plots

# Initialize publication-quality plotting defaults
plots.set_style()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Data Sources and Sample Construction

### 2.1 Data Infrastructure

The analysis utilizes a modular data access layer (`ds.data`) that standardizes retrieval and caching of financial information. This approach ensures consistency across multiple analytical sessions and enables efficient peer group comparisons.

In [76]:
# =============================================================================
# Data Acquisition
# =============================================================================

# Define analysis universe
ticker = "BMW.DE"
peers = ["MBG.DE", "VOW3.DE", "P911.DE"]  # Mercedes-Benz, Volkswagen, Porsche
data_start_date = "2010-01-01"

# Primary Subject: BMW AG
df_prices = data.get_stock_data(ticker, start=data_start_date)
financials = data.get_company_financials(ticker)
quarterly_financials = data.get_quarterly_financials(ticker)
info = data.get_company_info(ticker)
extra_data = data.get_holders_and_recommendations(ticker)

# Peer Group
peer_data = {}
for p in peers:
    peer_data[p] = {
        "info": data.get_company_info(p),
        "financials": data.get_company_financials(p),
        "prices": data.get_stock_data(p, start=data_start_date),
    }

# Risk-Free Rate Proxy (US 10Y Treasury)
risk_free_rate = data.get_treasury_yield("^TNX", start=data_start_date)

# Summary table
summary_data = {
    "Metric": ["Company", "Sector", "Beta", "Market Cap (€B)", "Data Range"],
    "Value": [
        info.get("shortName", ticker),
        info.get("sector"),
        f"{info.get('beta'):.2f}",
        f"€{info.get('marketCap', 0) / 1e9:,.1f}B",
        f"{df_prices.index.min():%Y-%m-%d} to {df_prices.index.max():%Y-%m-%d}",
    ],
}
display(pd.DataFrame(summary_data).style.hide(axis="index").set_caption("Data Acquisition Summary"))

Loading BMW.DE from cache...
Loading BMW.DE income_stmt from cache...
Loading BMW.DE balance_sheet from cache...
Loading BMW.DE cashflow from cache...
Loading BMW.DE quarterly income_stmt from cache...
Loading BMW.DE quarterly balance_sheet from cache...
Loading BMW.DE quarterly cashflow from cache...
Loading BMW.DE info from cache...
Loading BMW.DE institutional_holders from cache...
Loading BMW.DE major_holders from cache...
Loading BMW.DE recommendations from cache...
Loading MBG.DE info from cache...
Loading MBG.DE income_stmt from cache...
Loading MBG.DE balance_sheet from cache...
Loading MBG.DE cashflow from cache...
Loading MBG.DE from cache...
Loading VOW3.DE info from cache...
Loading VOW3.DE income_stmt from cache...
Loading VOW3.DE balance_sheet from cache...
Loading VOW3.DE cashflow from cache...
Loading VOW3.DE from cache...
Loading P911.DE info from cache...
Loading P911.DE income_stmt from cache...
Loading P911.DE balance_sheet from cache...
Loading P911.DE cashflow fro

Metric,Value
Company,BAYERISCHE MOTOREN WERKE AG S
Sector,Consumer Cyclical
Beta,0.77
Market Cap (€B),€51.9B
Data Range,2019-01-02 to 2025-11-21


### 2.2 Peer Group Selection

The comparative analysis employs a peer group of German premium automotive manufacturers. This selection criteria ensures consistency in:

- **Regulatory Environment**: EU emissions standards and labor regulations
- **Currency Exposure**: Euro-denominated operations and reporting
- **Market Positioning**: Premium and luxury segment competition

| Ticker | Company | Segment |
|--------|---------|---------|
| MBG.DE | Mercedes-Benz Group AG | Premium full-line |
| VOW3.DE | Volkswagen AG | Mass-market with premium subsidiaries |
| P911.DE | Porsche AG | Luxury/Performance |

### 2.3 Data Elements

The following data elements are extracted for the trailing five-year period:

1. **Market Data**: Daily Open-High-Low-Close-Volume (OHLCV) prices
2. **Income Statement**: Revenue, EBIT, Tax Provision, Net Income
3. **Balance Sheet**: Total Equity, Total Debt, Cash and Equivalents
4. **Cash Flow Statement**: Operating Cash Flow, Capital Expenditures
5. **Reference Rate**: US 10-Year Treasury Yield as risk-free rate proxy

## 3. Methodology: Financial Metrics Computation

### 3.1 Key Performance Indicators

The valuation framework relies on several derived metrics computed from raw financial statement data:

| Metric | Formula | Interpretation |
|--------|---------|----------------|
| **Free Cash Flow (FCF)** | OCF − CapEx | Cash available to capital providers |
| **NOPAT** | EBIT × (1 − Tax Rate) | Operating profit after tax adjustment |
| **Invested Capital** | Equity + Debt − Cash | Capital deployed in operations |
| **ROIC** | NOPAT / Invested Capital | Return on capital employed |
| **EBIT Margin** | EBIT / Revenue | Operating profitability |

### 3.2 Implementation

The following code block transforms raw financial statement data into the standardized metrics defined above. Column name matching is handled flexibly to accommodate variations in reporting conventions.

In [77]:
# =============================================================================
# Financial Metrics Computation
# =============================================================================


def get_col(df: pd.DataFrame, keywords: list[str]) -> pd.Series:
    """Retrieve a column from a DataFrame by matching partial keywords."""
    for col in df.columns:
        if any(k in col for k in keywords):
            return df[col]
    return pd.Series(0, index=df.index)


def prepare_financial_history(financials_dict: dict[str, pd.DataFrame]) -> pd.DataFrame:
    """Transform raw financial statements into standardized analytical metrics."""
    income_T = financials_dict["income_stmt"].T
    balance_T = financials_dict["balance_sheet"].T
    cashflow_T = financials_dict["cashflow"].T

    df_hist = pd.DataFrame(index=income_T.index)

    # Income Statement Items
    df_hist["Revenue"] = get_col(income_T, ["Total Revenue", "Operating Revenue"])
    df_hist["EBIT"] = get_col(income_T, ["EBIT", "Operating Income"])
    df_hist["Pretax Income"] = get_col(income_T, ["Pretax Income"])
    df_hist["Tax Provision"] = get_col(income_T, ["Tax Provision", "Income Tax Expense"])
    df_hist["NetIncome"] = get_col(income_T, ["Net Income", "Net Income Common Stockholders"])

    # Balance Sheet Items
    df_hist["Total Equity"] = get_col(
        balance_T, ["Total Stockholder Equity", "Total Equity Gross Minority Interest"]
    )
    df_hist["Total Debt"] = get_col(balance_T, ["Total Debt"])
    df_hist["Cash"] = get_col(balance_T, ["Cash And Cash Equivalents"])

    # Cash Flow Items
    df_hist["OCF"] = get_col(
        cashflow_T, ["Operating Cash Flow", "Total Cash From Operating Activities"]
    )
    df_hist["CapEx"] = get_col(cashflow_T, ["Capital Expenditure"])

    # Derived Metrics
    if df_hist["CapEx"].mean() > 0:
        df_hist["FCF"] = df_hist["OCF"] - df_hist["CapEx"]
    else:
        df_hist["FCF"] = df_hist["OCF"] + df_hist["CapEx"]

    df_hist["Tax Rate"] = (df_hist["Tax Provision"] / df_hist["Pretax Income"]).fillna(0.25)
    df_hist["NOPAT"] = df_hist["EBIT"] * (1 - df_hist["Tax Rate"])
    df_hist["Invested Capital"] = df_hist["Total Equity"] + df_hist["Total Debt"] - df_hist["Cash"]
    df_hist["ROIC"] = df_hist["NOPAT"] / df_hist["Invested Capital"]
    df_hist["EBIT Margin"] = df_hist["EBIT"] / df_hist["Revenue"]
    df_hist["FCF Margin"] = df_hist["FCF"] / df_hist["Revenue"]

    return df_hist.sort_index()


# Apply Transformations
df_annual = prepare_financial_history(financials)
df_quarterly = prepare_financial_history(quarterly_financials)

peer_metrics = {}
for p, p_data in peer_data.items():
    peer_metrics[p] = prepare_financial_history(p_data["financials"])

# Display Summary Table
display(
    df_annual[["Revenue", "EBIT", "NetIncome", "FCF", "ROIC"]]
    .tail()
    .style.format(
        {
            "Revenue": "€{:,.0f}",
            "EBIT": "€{:,.0f}",
            "NetIncome": "€{:,.0f}",
            "FCF": "€{:,.0f}",
            "ROIC": "{:.1%}",
        }
    )
    .set_caption("Table 1: BMW AG Annual Financial Summary")
)

,Revenue,EBIT,NetIncome,FCF,ROIC
2020-12-31 00:00:00,€nan,€nan,€nan,€nan,nan%
2021-12-31 00:00:00,"€111,239,000,000","€22,287,000,000","€12,382,000,000","€9,295,000,000",12.1%
2022-12-31 00:00:00,"€142,610,000,000","€31,362,000,000","€17,941,000,000","€14,473,000,000",17.0%
2023-12-31 00:00:00,"€155,498,000,000","€27,963,000,000","€11,290,000,000","€6,661,000,000",13.5%
2024-12-31 00:00:00,"€142,380,000,000","€20,783,000,000","€7,290,000,000","€-4,639,000,000",9.0%


In [78]:
# =============================================================================
# Data Preparation for Visualization
# =============================================================================
# Convert index to DatetimeIndex and extract fiscal years for plotting
df_annual.index = pd.to_datetime(df_annual.index)
years = pd.DatetimeIndex(df_annual.index).year

## 4. Results: Historical Performance Analysis

### 4.1 Dashboard Overview

Figure 1 presents a multi-panel visualization of key financial metrics. The dashboard structure facilitates comparison across dimensions:

| Panel | Metric | Purpose |
|-------|--------|---------|
| (1,1) | Revenue & EBIT Margin | Scale and operating leverage relationship |
| (1,2) | FCF Composition | Cash generation capacity vs. reinvestment |
| (2,1) | Quarterly Revenue | Seasonality and trend identification |
| (2,2) | Stock Price | Market valuation trajectory |
| (3,1) | ROIC | Capital efficiency over time |
| (3,2) | Peer Comparison | Relative operating performance |

In [79]:
# =============================================================================
# Figure 1: Financial Performance Dashboard
# =============================================================================

fig = make_subplots(
    rows=3,
    cols=2,
    subplot_titles=(
        "(a) Revenue & EBIT Margin",
        "(b) Free Cash Flow Composition",
        "(c) Quarterly Revenue Trend",
        "(d) Stock Price History",
        "(e) Return on Invested Capital",
        "(f) Peer Comparison: Operating Margin",
    ),
    specs=[
        [{"secondary_y": True}, {"secondary_y": False}],
        [{"secondary_y": False}, {"secondary_y": False}],
        [{"secondary_y": False}, {"secondary_y": False}],
    ],
    vertical_spacing=0.10,
)

# -----------------------------------------------------------------------------
# Panel (a): Revenue & EBIT Margin
# -----------------------------------------------------------------------------
fig.add_trace(
    go.Bar(
        x=years,
        y=df_annual["Revenue"],
        name="Revenue",
        marker_color="rgb(55, 83, 109)",
    ),
    row=1,
    col=1,
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(
        x=years,
        y=df_annual["EBIT Margin"],
        name="EBIT Margin",
        mode="lines+markers",
        line=dict(color="rgb(26, 118, 255)", width=3),
    ),
    row=1,
    col=1,
    secondary_y=True,
)

# -----------------------------------------------------------------------------
# Panel (b): FCF Composition
# -----------------------------------------------------------------------------
fig.add_trace(
    go.Bar(
        x=years,
        y=df_annual["OCF"],
        name="Operating Cash Flow",
        marker_color="rgb(44, 160, 101)",
    ),
    row=1,
    col=2,
)
fig.add_trace(
    go.Bar(
        x=years,
        y=df_annual["CapEx"],
        name="Capital Expenditure",
        marker_color="rgb(214, 39, 40)",
    ),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(
        x=years,
        y=df_annual["FCF"],
        name="Free Cash Flow",
        line=dict(color="black", width=3, dash="dash"),
    ),
    row=1,
    col=2,
)

# -----------------------------------------------------------------------------
# Panel (c): Quarterly Revenue
# -----------------------------------------------------------------------------
fig.add_trace(
    go.Scatter(
        x=df_quarterly.index,
        y=df_quarterly["Revenue"],
        name="Quarterly Revenue",
        fill="tozeroy",
        line=dict(color="rgb(148, 103, 189)"),
    ),
    row=2,
    col=1,
)

# -----------------------------------------------------------------------------
# Panel (d): Stock Price
# -----------------------------------------------------------------------------
fig.add_trace(
    go.Scatter(
        x=df_prices.index,
        y=df_prices["Close"],
        name="Closing Price",
        line=dict(color="rgb(255, 127, 14)"),
    ),
    row=2,
    col=2,
)

# -----------------------------------------------------------------------------
# Panel (e): ROIC
# -----------------------------------------------------------------------------
fig.add_trace(
    go.Bar(
        x=years,
        y=df_annual["ROIC"],
        name="ROIC",
        marker_color="rgb(31, 119, 180)",
    ),
    row=3,
    col=1,
)

# -----------------------------------------------------------------------------
# Panel (f): Peer Comparison
# -----------------------------------------------------------------------------
fig.add_trace(
    go.Scatter(
        x=years,
        y=df_annual["EBIT Margin"],
        name=f"{ticker}",
        mode="lines+markers",
        line=dict(width=3),
    ),
    row=3,
    col=2,
)

for p, p_df in peer_metrics.items():
    p_years = pd.DatetimeIndex(p_df.index).year
    fig.add_trace(
        go.Scatter(
            x=p_years,
            y=p_df["EBIT Margin"],
            name=p,
            mode="lines",
            line=dict(dash="dot"),
        ),
        row=3,
        col=2,
    )

# -----------------------------------------------------------------------------
# Layout Configuration
# -----------------------------------------------------------------------------
fig.update_layout(
    height=1200,
    title_text=f"Figure 1: Financial Performance Dashboard — {ticker}",
    template="plotly_white",
    showlegend=True,
    font=dict(size=11),
)

# Axis formatting
fig.update_yaxes(title_text="Revenue (EUR)", row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="Margin", tickformat=".1%", row=1, col=1, secondary_y=True)
fig.update_yaxes(title_text="Amount (EUR)", row=1, col=2)
fig.update_yaxes(title_text="Revenue (EUR)", row=2, col=1)
fig.update_yaxes(title_text="Price (EUR)", row=2, col=2)
fig.update_yaxes(title_text="ROIC", tickformat=".1%", row=3, col=1)
fig.update_yaxes(title_text="EBIT Margin", tickformat=".1%", row=3, col=2)

fig.show()

## 5. DCF Valuation Model

### 5.1 Weighted Average Cost of Capital (WACC)

The discount rate for BMW's cash flows is computed using the Weighted Average Cost of Capital:

$$\text{WACC} = w_E \cdot r_e + w_D \cdot r_d \cdot (1-t)$$

**Cost of Equity (CAPM):**
$$r_e = R_f + \beta \cdot (R_m - R_f)$$

#### Assumed Values

| Parameter | Value | Rationale |
|-----------|-------|-----------|
| Equity Risk Premium $(R_m - R_f)$ | **5.5%** | Industry standard for European equities (5-6%); elevated for cyclical auto sector |
| Pre-tax Cost of Debt $(r_d)$ | **4.5%** | BMW investment-grade credit rating (A/A2); current corporate bond yields |
| Tax Rate $(t)$ | **30%** | German statutory (~15%) + trade tax (~14%) |
| Target Debt Weight $(w_D)$ | **30%** | Target capital structure for industrial operations; excludes captive financing |
| Beta Floor | **1.0** | Cyclical auto industry warrants minimum beta (raw beta often understates risk) |

#### Data from Source


| Parameter | Source || Market Cap, Debt, Cash | Latest annual financial statements |

|-----------|--------|| Reported Beta | Yahoo Finance company info |
| Risk-Free Rate $(R_f)$ | 10Y Treasury yield (^TNX), most recent |

In [80]:
# =============================================================================
# WACC Calculation (Adjusted for Auto Industry)
# =============================================================================
import numpy as np

# Risk-Free Rate
rf = risk_free_rate.iloc[-1] / 100

# Beta (floor at 1.0 for cyclical auto industry)
reported_beta = info.get("beta", 1.0)
beta = max(reported_beta, 1.0)

# Equity Risk Premium (slightly higher for European autos)
equity_risk_premium = 0.055

# Cost of Equity (CAPM)
cost_of_equity = rf + beta * equity_risk_premium

# Cost of Debt
cost_of_debt_pretax = 0.045
tax_rate = 0.30
cost_of_debt_aftertax = cost_of_debt_pretax * (1 - tax_rate)

# Capital Structure (reported)
market_cap = info.get("marketCap", 0)
total_debt = df_annual["Total Debt"].iloc[-1]
cash = df_annual["Cash"].iloc[-1]
net_debt = total_debt - cash
enterprise_value = market_cap + net_debt

# Target Capital Structure (industrial operations)
TARGET_DEBT_TO_EV = 0.30
weight_equity = 1 - TARGET_DEBT_TO_EV
weight_debt = TARGET_DEBT_TO_EV

# WACC Calculation
wacc = weight_equity * cost_of_equity + weight_debt * cost_of_debt_aftertax

# Store for later use
dcf_params = {
    "rf": rf,
    "beta": beta,
    "equity_risk_premium": equity_risk_premium,
    "cost_of_equity": cost_of_equity,
    "cost_of_debt_pretax": cost_of_debt_pretax,
    "cost_of_debt_aftertax": cost_of_debt_aftertax,
    "tax_rate": tax_rate,
    "market_cap": market_cap,
    "total_debt": total_debt,
    "cash": cash,
    "net_debt": net_debt,
    "enterprise_value": enterprise_value,
    "weight_equity": weight_equity,
    "weight_debt": weight_debt,
    "wacc": wacc,
}

# Display WACC Summary Table
wacc_data = pd.DataFrame(
    {
        "Component": [
            "Risk-Free Rate",
            "Beta (adjusted)",
            "Equity Risk Premium",
            "Cost of Equity",
            "Cost of Debt (after-tax)",
            "Weight Equity",
            "Weight Debt",
            "WACC",
        ],
        "Value": [
            f"{rf:.2%}",
            f"{beta:.2f}",
            f"{equity_risk_premium:.2%}",
            f"{cost_of_equity:.2%}",
            f"{cost_of_debt_aftertax:.2%}",
            f"{weight_equity:.1%}",
            f"{weight_debt:.1%}",
            f"{wacc:.2%}",
        ],
    }
)
display(wacc_data.style.hide(axis="index").set_caption("Table 2: WACC Components"))

Component,Value
Risk-Free Rate,4.06%
Beta (adjusted),1.00
Equity Risk Premium,5.50%
Cost of Equity,9.56%
Cost of Debt (after-tax),3.15%
Weight Equity,70.0%
Weight Debt,30.0%
WACC,7.64%


In [81]:
# =============================================================================
# WACC Component Visualization
# =============================================================================

fig_wacc = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("(a) Capital Structure", "(b) WACC Components"),
    specs=[[{"type": "pie"}, {"type": "bar"}]],
)

# Panel (a): Capital Structure Pie Chart
fig_wacc.add_trace(
    go.Pie(
        labels=["Equity", "Net Debt"],
        values=[market_cap, net_debt],
        marker_colors=["rgb(55, 83, 109)", "rgb(26, 118, 255)"],
        textinfo="label+percent",
        textposition="inside",
        hole=0.4,
    ),
    row=1,
    col=1,
)

# Panel (b): WACC Components Bar Chart
components = ["Cost of Equity", "Cost of Debt\n(after-tax)", "WACC"]
values = [cost_of_equity * 100, cost_of_debt_aftertax * 100, wacc * 100]
colors = ["rgb(44, 160, 101)", "rgb(214, 39, 40)", "rgb(148, 103, 189)"]

fig_wacc.add_trace(
    go.Bar(
        x=components,
        y=values,
        marker_color=colors,
        text=[f"{v:.2f}%" for v in values],
        textposition="outside",
    ),
    row=1,
    col=2,
)

fig_wacc.update_layout(
    height=450,
    title_text=f"Figure 2: WACC Analysis — {ticker}",
    template="plotly_white",
    showlegend=False,
    margin=dict(t=80, b=60, l=60, r=60),
)

fig_wacc.update_yaxes(title_text="Rate (%)", row=1, col=2, range=[0, 12])

fig_wacc.show()

# Summary Table
print("\nTable 2: WACC Summary")
print("-" * 50)
wacc_summary = pd.DataFrame(
    {
        "Parameter": [
            "Risk-Free Rate (Rf)",
            "Beta (β)",
            "Equity Risk Premium",
            "Cost of Equity (r_e)",
            "Cost of Debt (pre-tax)",
            "Tax Rate",
            "Cost of Debt (after-tax)",
            "Weight of Equity",
            "Weight of Debt",
            "WACC",
        ],
        "Value": [
            f"{rf:.2%}",
            f"{beta:.2f}",
            f"{equity_risk_premium:.2%}",
            f"{cost_of_equity:.2%}",
            f"{cost_of_debt_pretax:.2%}",
            f"{tax_rate:.0%}",
            f"{cost_of_debt_aftertax:.2%}",
            f"{weight_equity:.1%}",
            f"{weight_debt:.1%}",
            f"{wacc:.2%}",
        ],
    }
)
display(wacc_summary.style.hide(axis="index"))


Table 2: WACC Summary
--------------------------------------------------


Parameter,Value
Risk-Free Rate (Rf),4.06%
Beta (β),1.00
Equity Risk Premium,5.50%
Cost of Equity (r_e),9.56%
Cost of Debt (pre-tax),4.50%
Tax Rate,30%
Cost of Debt (after-tax),3.15%
Weight of Equity,70.0%
Weight of Debt,30.0%
WACC,7.64%


### 5.2 Five-Year Free Cash Flow Forecast (2025–2029)

The explicit forecast period projects BMW's financial performance using conservative assumptions.

#### Assumed Values

| Parameter | Y1 | Y2 | Y3 | Y4 | Y5 | Rationale |
|-----------|-----|-----|-----|-----|-----|-----------|
| Revenue Growth | **1%** | **2%** | **2%** | **2%** | **2%** | Conservative; reflects EV transition headwinds and competitive pressure |
| EBIT Margin | **8%** | **8.5%** | **9%** | **9%** | **9%** | Historical BMW margins 8-12%; modest recovery assumed |
| CapEx/Revenue | **8.5%** | **8%** | **7.5%** | **7%** | **6.5%** | Elevated EV investment cycle; gradual normalization |
| D&A/Revenue | **6%** | **6%** | **6%** | **6%** | **6%** | Higher than peers due to EV-related asset base |
| ΔNWC/ΔRevenue | **10%** | **10%** | **10%** | **10%** | **10%** | Industry standard working capital intensity |
| Tax Rate | **30%** | — | — | — | — | German statutory rate (assumed constant) |

**Free Cash Flow Formula:**
$$\text{FCF}_t = \text{NOPAT}_t + \text{D\&A}_t - \text{CapEx}_t - \Delta\text{NWC}_t$$

In [82]:
# =============================================================================
# 5-Year Free Cash Flow Projection (2025-2029)
# =============================================================================

# Base Year Data
base_year = df_annual.index[-1].year
base_revenue = df_annual["Revenue"].iloc[-1]
base_ebit_margin = df_annual["EBIT Margin"].iloc[-1]

# Forecast Assumptions (conservative, based on historical data)
forecast_years = [base_year + i for i in range(1, 6)]
revenue_growth = [0.01, 0.02, 0.02, 0.02, 0.02]
ebit_margin_forecast = [0.08, 0.085, 0.09, 0.09, 0.09]
capex_pct = [0.085, 0.080, 0.075, 0.070, 0.065]
da_pct = [0.06, 0.06, 0.06, 0.06, 0.06]
nwc_pct_of_delta_rev = 0.10
forecast_tax_rate = tax_rate

# Build Forecast Model
forecast_data = []
prev_revenue = base_revenue

for i, year in enumerate(forecast_years):
    revenue = prev_revenue * (1 + revenue_growth[i])
    delta_revenue = revenue - prev_revenue
    ebit = revenue * ebit_margin_forecast[i]
    nopat = ebit * (1 - forecast_tax_rate)
    da = revenue * da_pct[i]
    capex = revenue * capex_pct[i]
    delta_nwc = delta_revenue * nwc_pct_of_delta_rev
    fcf = nopat + da - capex - delta_nwc

    forecast_data.append(
        {
            "Year": year,
            "Revenue": revenue,
            "Revenue Growth": revenue_growth[i],
            "EBIT": ebit,
            "EBIT Margin": ebit_margin_forecast[i],
            "NOPAT": nopat,
            "D&A": da,
            "CapEx": capex,
            "ΔNWC": delta_nwc,
            "FCF": fcf,
            "FCF Margin": fcf / revenue,
        }
    )
    prev_revenue = revenue

df_forecast = pd.DataFrame(forecast_data).set_index("Year")

# =============================================================================
# Visualize Forecast Assumptions (replaces Table 3)
# =============================================================================
fig_assumptions = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("(a) Revenue & Margin Trajectory", "(b) Capital Intensity"),
    specs=[[{"secondary_y": True}, {"secondary_y": True}]],
)

# Panel (a): Revenue Growth and EBIT Margin
fig_assumptions.add_trace(
    go.Bar(
        x=[str(y) for y in forecast_years],
        y=[g * 100 for g in revenue_growth],
        name="Revenue Growth",
        marker_color="rgb(55, 83, 109)",
        text=[f"{g:.1%}" for g in revenue_growth],
        textposition="outside",
    ),
    row=1,
    col=1,
    secondary_y=False,
)

fig_assumptions.add_trace(
    go.Scatter(
        x=[str(y) for y in forecast_years],
        y=[m * 100 for m in ebit_margin_forecast],
        name="EBIT Margin",
        mode="lines+markers+text",
        line=dict(color="rgb(44, 160, 101)", width=3),
        marker=dict(size=10),
        text=[f"{m:.1%}" for m in ebit_margin_forecast],
        textposition="top center",
    ),
    row=1,
    col=1,
    secondary_y=True,
)

# Panel (b): CapEx and D&A
fig_assumptions.add_trace(
    go.Scatter(
        x=[str(y) for y in forecast_years],
        y=[c * 100 for c in capex_pct],
        name="CapEx/Rev",
        mode="lines+markers",
        line=dict(color="rgb(214, 39, 40)", width=3),
        marker=dict(size=10),
    ),
    row=1,
    col=2,
    secondary_y=False,
)

fig_assumptions.add_trace(
    go.Scatter(
        x=[str(y) for y in forecast_years],
        y=[d * 100 for d in da_pct],
        name="D&A/Rev",
        mode="lines+markers",
        line=dict(color="rgb(148, 103, 189)", width=3, dash="dash"),
        marker=dict(size=10),
    ),
    row=1,
    col=2,
    secondary_y=False,
)

fig_assumptions.update_layout(
    height=350,
    title_text=f"Forecast Assumptions (2025–2029) — {ticker}",
    template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=-0.25, xanchor="center", x=0.5),
    margin=dict(t=80, b=80),
)

fig_assumptions.update_yaxes(
    title_text="Revenue Growth (%)", row=1, col=1, secondary_y=False, range=[0, 5]
)
fig_assumptions.update_yaxes(
    title_text="EBIT Margin (%)", row=1, col=1, secondary_y=True, range=[6, 12]
)
fig_assumptions.update_yaxes(title_text="% of Revenue", row=1, col=2, range=[4, 10])

fig_assumptions.show()

# Store for later use
dcf_params["df_forecast"] = df_forecast
dcf_params["base_revenue"] = base_revenue
dcf_params["terminal_fcf"] = df_forecast["FCF"].iloc[-1]

In [83]:
# =============================================================================
# Figure 3: FCF Forecast Visualization
# =============================================================================

fig_forecast = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=(
        "(a) Revenue & EBIT Margin Forecast",
        "(b) Free Cash Flow Projection",
    ),
    specs=[[{"secondary_y": True}, {"secondary_y": False}]],
)

# Historical + Forecast years
hist_years = list(years[-3:])  # Last 3 historical years
fcst_years = list(df_forecast.index)
all_years = hist_years + fcst_years

# Historical data (last 3 years)
hist_revenue = df_annual["Revenue"].iloc[-3:].values / 1e9
hist_ebit_margin = df_annual["EBIT Margin"].iloc[-3:].values
hist_fcf = df_annual["FCF"].iloc[-3:].values / 1e9

# Forecast data
fcst_revenue = df_forecast["Revenue"].values / 1e9
fcst_ebit_margin = df_forecast["EBIT Margin"].values
fcst_fcf = df_forecast["FCF"].values / 1e9

# -----------------------------------------------------------------------------
# Panel (a): Revenue & EBIT Margin
# -----------------------------------------------------------------------------
# Historical Revenue
fig_forecast.add_trace(
    go.Bar(
        x=hist_years,
        y=hist_revenue,
        name="Revenue (Historical)",
        marker_color="rgb(55, 83, 109)",
    ),
    row=1,
    col=1,
    secondary_y=False,
)

# Forecast Revenue
fig_forecast.add_trace(
    go.Bar(
        x=fcst_years,
        y=fcst_revenue,
        name="Revenue (Forecast)",
        marker_color="rgba(55, 83, 109, 0.5)",
        marker_line=dict(color="rgb(55, 83, 109)", width=2),
    ),
    row=1,
    col=1,
    secondary_y=False,
)

# EBIT Margin line
fig_forecast.add_trace(
    go.Scatter(
        x=all_years,
        y=list(hist_ebit_margin) + list(fcst_ebit_margin),
        name="EBIT Margin",
        mode="lines+markers",
        line=dict(color="rgb(26, 118, 255)", width=3),
        marker=dict(size=10),
    ),
    row=1,
    col=1,
    secondary_y=True,
)

# Divider line between historical and forecast
fig_forecast.add_vline(
    x=hist_years[-1] + 0.5,
    line_dash="dash",
    line_color="gray",
    row=1,
    col=1,
)

# -----------------------------------------------------------------------------
# Panel (b): FCF Waterfall-style
# -----------------------------------------------------------------------------
# Historical FCF
fig_forecast.add_trace(
    go.Bar(
        x=hist_years,
        y=hist_fcf,
        name="FCF (Historical)",
        marker_color="rgb(44, 160, 101)",
    ),
    row=1,
    col=2,
)

# Forecast FCF
fig_forecast.add_trace(
    go.Bar(
        x=fcst_years,
        y=fcst_fcf,
        name="FCF (Forecast)",
        marker_color="rgba(44, 160, 101, 0.5)",
        marker_line=dict(color="rgb(44, 160, 101)", width=2),
    ),
    row=1,
    col=2,
)

# Divider line
fig_forecast.add_vline(
    x=hist_years[-1] + 0.5,
    line_dash="dash",
    line_color="gray",
    row=1,
    col=2,
)

# Add annotation for forecast period
fig_forecast.add_annotation(
    x=fcst_years[2],
    y=max(fcst_fcf) * 1.1,
    text="Forecast Period",
    showarrow=False,
    font=dict(size=12, color="gray"),
    row=1,
    col=2,
)

# -----------------------------------------------------------------------------
# Layout
# -----------------------------------------------------------------------------
fig_forecast.update_layout(
    height=450,
    title_text=f"Figure 3: 5-Year Financial Forecast — {ticker}",
    template="plotly_white",
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=-0.25, xanchor="center", x=0.5),
    barmode="group",
)

fig_forecast.update_yaxes(title_text="Revenue (€B)", row=1, col=1, secondary_y=False)
fig_forecast.update_yaxes(
    title_text="EBIT Margin", tickformat=".0%", row=1, col=1, secondary_y=True
)
fig_forecast.update_yaxes(title_text="FCF (€B)", row=1, col=2)

fig_forecast.show()

# -----------------------------------------------------------------------------
# FCF Bridge Chart
# -----------------------------------------------------------------------------
print("\nTable 3: FCF Component Bridge (Terminal Year 2029)")
print("-" * 60)

terminal_year = df_forecast.iloc[-1]
bridge_data = pd.DataFrame(
    {
        "Component": ["NOPAT", "+ D&A", "− CapEx", "− ΔNWC", "= FCF"],
        "Amount (€B)": [
            terminal_year["NOPAT"] / 1e9,
            terminal_year["D&A"] / 1e9,
            -terminal_year["CapEx"] / 1e9,
            -terminal_year["ΔNWC"] / 1e9,
            terminal_year["FCF"] / 1e9,
        ],
    }
)
display(bridge_data.style.format({"Amount (€B)": "{:,.2f}"}).hide(axis="index"))


Table 3: FCF Component Bridge (Terminal Year 2029)
------------------------------------------------------------


Component,Amount (€B)
NOPAT,9.81
+ D&A,9.34
− CapEx,-10.12
− ΔNWC,-0.31
= FCF,8.72


### 5.3 Terminal Value

Beyond the explicit forecast period, we estimate BMW's continuing value using the **Gordon Growth Model** (perpetuity growth method):

$$TV_{2029} = \frac{FCF_{2029} \times (1 + g)}{WACC - g}$$

Where:
- $FCF_{2029}$ = Terminal year free cash flow
- $g$ = Perpetual growth rate (2%, approximating long-run GDP growth)
- $WACC$ = Weighted average cost of capital

**Cross-Check:** The implied exit multiple should fall within reasonable industry ranges:
- EV/EBITDA: 6–10× for mature automakers
- EV/FCF: 12–18× for stable cash-generative businesses

In [84]:
# =============================================================================
# Terminal Value Calculation
# =============================================================================

# -----------------------------------------------------------------------------
# Terminal Value Assumptions
# -----------------------------------------------------------------------------
terminal_growth_rate = 0.02  # 2% perpetual growth (long-run GDP proxy)

# Terminal year FCF
terminal_fcf = df_forecast["FCF"].iloc[-1]
terminal_year_num = df_forecast.index[-1]

print("=" * 60)
print("TERMINAL VALUE CALCULATION")
print("=" * 60)
print(f"\nTerminal Year: {terminal_year_num}")
print(f"Terminal FCF: €{terminal_fcf / 1e9:,.2f}B")
print(f"Perpetual Growth Rate (g): {terminal_growth_rate:.1%}")
print(f"WACC: {wacc:.2%}")

# -----------------------------------------------------------------------------
# Gordon Growth Model: TV = FCF × (1+g) / (WACC - g)
# -----------------------------------------------------------------------------
terminal_value = (terminal_fcf * (1 + terminal_growth_rate)) / (wacc - terminal_growth_rate)

print("\nTerminal Value Formula:")
print("  TV = FCF₂₀₂₉ × (1 + g) / (WACC - g)")
print(
    f"  TV = €{terminal_fcf / 1e9:,.2f}B × (1 + {terminal_growth_rate:.1%}) / ({wacc:.2%} - {terminal_growth_rate:.1%})"
)
print(
    f"  TV = €{terminal_fcf * (1 + terminal_growth_rate) / 1e9:,.2f}B / {wacc - terminal_growth_rate:.2%}"
)
print(f"\n  Terminal Value = €{terminal_value / 1e9:,.1f}B")

# -----------------------------------------------------------------------------
# Implied Exit Multiples (Sanity Check)
# -----------------------------------------------------------------------------
terminal_ebitda = df_forecast["EBIT"].iloc[-1] + df_forecast["D&A"].iloc[-1]
implied_ev_ebitda = terminal_value / terminal_ebitda
implied_ev_fcf = terminal_value / terminal_fcf

print("\nImplied Exit Multiples (Sanity Check):")
print(f"  Terminal EBITDA: €{terminal_ebitda / 1e9:,.2f}B")
print(f"  Implied EV/EBITDA: {implied_ev_ebitda:.1f}×")
print(f"  Implied EV/FCF: {implied_ev_fcf:.1f}×")

if 6 <= implied_ev_ebitda <= 12:
    print("  ✓ EV/EBITDA within reasonable range (6-12×)")
else:
    print("  ⚠ EV/EBITDA outside typical range (6-12×)")

# Store for later use
dcf_params["terminal_growth_rate"] = terminal_growth_rate
dcf_params["terminal_value"] = terminal_value
dcf_params["implied_ev_ebitda"] = implied_ev_ebitda

TERMINAL VALUE CALCULATION

Terminal Year: 2029
Terminal FCF: €8.72B
Perpetual Growth Rate (g): 2.0%
WACC: 7.64%

Terminal Value Formula:
  TV = FCF₂₀₂₉ × (1 + g) / (WACC - g)
  TV = €8.72B × (1 + 2.0%) / (7.64% - 2.0%)
  TV = €8.90B / 5.64%

  Terminal Value = €157.8B

Implied Exit Multiples (Sanity Check):
  Terminal EBITDA: €23.35B
  Implied EV/EBITDA: 6.8×
  Implied EV/FCF: 18.1×
  ✓ EV/EBITDA within reasonable range (6-12×)


### 5.4 Present Value & Enterprise Value

The DCF valuation discounts all future cash flows back to present value:

$$EV = \sum_{t=1}^{5} \frac{FCF_t}{(1 + WACC)^t} + \frac{TV}{(1 + WACC)^5}$$

Where:
- First term = Present value of explicit forecast period FCFs
- Second term = Present value of terminal value

The **equity value bridge** then derives shareholder value:

$$\text{Equity Value} = EV - \text{Net Debt}$$

$$\text{Intrinsic Value per Share} = \frac{\text{Equity Value}}{\text{Shares Outstanding}}$$

In [85]:
# =============================================================================
# Present Value & Enterprise Value Calculation
# =============================================================================

# -----------------------------------------------------------------------------
# Discount Factors
# -----------------------------------------------------------------------------
discount_factors = [(1 + wacc) ** t for t in range(1, 6)]

print("=" * 70)
print("DCF VALUATION: PRESENT VALUE CALCULATION")
print("=" * 70)

# -----------------------------------------------------------------------------
# Present Value of Explicit Period FCFs
# -----------------------------------------------------------------------------
pv_fcfs = []
print("\nPresent Value of Forecast Period FCFs:")
print("-" * 70)
print(f"{'Year':<8} {'FCF (€B)':<12} {'Discount Factor':<18} {'PV (€B)':<12}")
print("-" * 70)

for i, (year, row) in enumerate(df_forecast.iterrows()):
    fcf_t = row["FCF"]
    df_t = discount_factors[i]
    pv_t = fcf_t / df_t
    pv_fcfs.append(pv_t)
    print(f"{year:<8} {fcf_t / 1e9:>10,.2f}   {df_t:>16,.4f}   {pv_t / 1e9:>10,.2f}")

pv_forecast_fcfs = sum(pv_fcfs)
print("-" * 70)
print(f"{'Total PV of FCFs':<40} {pv_forecast_fcfs / 1e9:>10,.2f}")

# -----------------------------------------------------------------------------
# Present Value of Terminal Value
# -----------------------------------------------------------------------------
pv_terminal_value = terminal_value / discount_factors[-1]

print("\nPresent Value of Terminal Value:")
print(f"  TV (end of {terminal_year_num}): €{terminal_value / 1e9:,.1f}B")
print(f"  Discount Factor (Year 5): {discount_factors[-1]:.4f}")
print(f"  PV of TV: €{pv_terminal_value / 1e9:,.1f}B")

# -----------------------------------------------------------------------------
# Enterprise Value
# -----------------------------------------------------------------------------
enterprise_value_dcf = pv_forecast_fcfs + pv_terminal_value

print("\n" + "=" * 70)
print("ENTERPRISE VALUE")
print("=" * 70)
print(f"  PV of Forecast FCFs:    €{pv_forecast_fcfs / 1e9:>10,.1f}B")
print(f"  PV of Terminal Value:   €{pv_terminal_value / 1e9:>10,.1f}B")
print("  ─────────────────────────────────────")
print(f"  Enterprise Value:       €{enterprise_value_dcf / 1e9:>10,.1f}B")

# TV as % of EV (sanity check)
tv_pct_of_ev = pv_terminal_value / enterprise_value_dcf
print(f"\n  Terminal Value as % of EV: {tv_pct_of_ev:.1%}")
if tv_pct_of_ev > 0.8:
    print("  ⚠ High TV concentration - valuation sensitive to terminal assumptions")

# Store for later
dcf_params["pv_forecast_fcfs"] = pv_forecast_fcfs
dcf_params["pv_terminal_value"] = pv_terminal_value
dcf_params["enterprise_value_dcf"] = enterprise_value_dcf

DCF VALUATION: PRESENT VALUE CALCULATION

Present Value of Forecast Period FCFs:
----------------------------------------------------------------------
Year     FCF (€B)     Discount Factor    PV (€B)     
----------------------------------------------------------------------
2025           4.32             1.0764         4.01
2026           5.51             1.1586         4.75
2027           6.89             1.2471         5.52
2028           7.79             1.3424         5.80
2029           8.72             1.4449         6.04
----------------------------------------------------------------------
Total PV of FCFs                              26.12

Present Value of Terminal Value:
  TV (end of 2029): €157.8B
  Discount Factor (Year 5): 1.4449
  PV of TV: €109.2B

ENTERPRISE VALUE
  PV of Forecast FCFs:    €      26.1B
  PV of Terminal Value:   €     109.2B
  ─────────────────────────────────────
  Enterprise Value:       €     135.3B

  Terminal Value as % of EV: 80.7%
  ⚠ High TV 

In [86]:
# =============================================================================
# Equity Value Bridge & Intrinsic Value
# =============================================================================

# -----------------------------------------------------------------------------
# Net Debt (from latest balance sheet)
# -----------------------------------------------------------------------------
net_debt_valuation = dcf_params["net_debt"]

# -----------------------------------------------------------------------------
# Shares Outstanding
# -----------------------------------------------------------------------------
shares_outstanding = info.get("sharesOutstanding", 600_000_000)

# -----------------------------------------------------------------------------
# Equity Value & Intrinsic Value per Share
# -----------------------------------------------------------------------------
equity_value = enterprise_value_dcf - net_debt_valuation
intrinsic_value_per_share = equity_value / shares_outstanding

# Current market price for comparison
current_price = df_prices["Close"].iloc[-1]

# Premium / Discount
premium_discount = (current_price - intrinsic_value_per_share) / intrinsic_value_per_share

print("\n" + "=" * 70)
print("EQUITY VALUE BRIDGE")
print("=" * 70)
print(f"  Enterprise Value (DCF):  €{enterprise_value_dcf / 1e9:>10,.1f}B")
print(f"  Less: Net Debt:          €{net_debt_valuation / 1e9:>10,.1f}B")
print("  ─────────────────────────────────────")
print(f"  Equity Value:            €{equity_value / 1e9:>10,.1f}B")

print(f"\n  Shares Outstanding:      {shares_outstanding / 1e6:,.0f}M")
print("  ─────────────────────────────────────")
print(f"  Intrinsic Value/Share:   €{intrinsic_value_per_share:>10,.2f}")

print("\n" + "=" * 70)
print("VALUATION SUMMARY")
print("=" * 70)
print(f"\n  DCF Intrinsic Value:     €{intrinsic_value_per_share:,.2f}")
print(f"  Current Market Price:    €{current_price:,.2f}")
print("  ─────────────────────────────────────")

if premium_discount > 0:
    print(f"  Market Premium:          {premium_discount:+.1%}")
    print("\n  → Stock appears OVERVALUED vs DCF estimate")
else:
    print(f"  Market Discount:         {premium_discount:+.1%}")
    print("\n  → Stock appears UNDERVALUED vs DCF estimate")

# Store final results
dcf_params["equity_value"] = equity_value
dcf_params["shares_outstanding"] = shares_outstanding
dcf_params["intrinsic_value_per_share"] = intrinsic_value_per_share
dcf_params["current_price"] = current_price
dcf_params["premium_discount"] = premium_discount


EQUITY VALUE BRIDGE
  Enterprise Value (DCF):  €     135.3B
  Less: Net Debt:          €      66.2B
  ─────────────────────────────────────
  Equity Value:            €      69.1B

  Shares Outstanding:      556M
  ─────────────────────────────────────
  Intrinsic Value/Share:   €    124.27

VALUATION SUMMARY

  DCF Intrinsic Value:     €124.27
  Current Market Price:    €85.08
  ─────────────────────────────────────
  Market Discount:         -31.5%

  → Stock appears UNDERVALUED vs DCF estimate


In [87]:
# =============================================================================
# Figure 4: DCF Valuation Summary Visualization
# =============================================================================

fig_dcf = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "(a) Enterprise Value Composition",
        "(b) Equity Value Bridge",
        "(c) Intrinsic vs Market Price",
        "(d) Valuation Metrics",
    ),
    specs=[
        [{"type": "pie"}, {"type": "waterfall"}],
        [{"type": "bar"}, {"type": "indicator"}],
    ],
    vertical_spacing=0.22,
    horizontal_spacing=0.12,
)

# -----------------------------------------------------------------------------
# Panel (a): Value Composition Donut
# -----------------------------------------------------------------------------
fig_dcf.add_trace(
    go.Pie(
        labels=["PV of FCFs<br>(2025-29)", "PV of<br>Terminal Value"],
        values=[pv_forecast_fcfs / 1e9, pv_terminal_value / 1e9],
        marker_colors=["rgb(44, 160, 101)", "rgb(148, 103, 189)"],
        textinfo="percent",
        textposition="inside",
        textfont=dict(size=14, color="white"),
        hole=0.5,
        hovertemplate="<b>%{label}</b><br>€%{value:.1f}B<br>%{percent}<extra></extra>",
    ),
    row=1,
    col=1,
)

# Add center annotation for total EV
fig_dcf.add_annotation(
    text=f"<b>EV</b><br>€{enterprise_value_dcf / 1e9:.0f}B",
    x=0.145,
    y=0.78,
    font=dict(size=14),
    showarrow=False,
    xref="paper",
    yref="paper",
)

# -----------------------------------------------------------------------------
# Panel (b): Equity Bridge Waterfall
# -----------------------------------------------------------------------------
fig_dcf.add_trace(
    go.Waterfall(
        x=["Enterprise<br>Value", "Less:<br>Net Debt", "Equity<br>Value"],
        y=[enterprise_value_dcf / 1e9, -net_debt_valuation / 1e9, equity_value / 1e9],
        measure=["absolute", "relative", "total"],
        text=[
            f"€{enterprise_value_dcf / 1e9:.0f}B",
            f"−€{net_debt_valuation / 1e9:.0f}B",
            f"€{equity_value / 1e9:.0f}B",
        ],
        textposition="outside",
        textfont=dict(size=12),
        connector={"line": {"color": "rgb(63, 63, 63)", "width": 1}},
        increasing={"marker": {"color": "rgb(44, 160, 101)"}},
        decreasing={"marker": {"color": "rgb(214, 39, 40)"}},
        totals={"marker": {"color": "rgb(55, 83, 109)"}},
    ),
    row=1,
    col=2,
)

# -----------------------------------------------------------------------------
# Panel (c): Price Comparison Bar
# -----------------------------------------------------------------------------
price_diff = intrinsic_value_per_share - current_price
bar_colors = ["rgb(55, 83, 109)", "rgb(44, 160, 101)" if price_diff > 0 else "rgb(214, 39, 40)"]

fig_dcf.add_trace(
    go.Bar(
        x=["Market Price", "DCF Value"],
        y=[current_price, intrinsic_value_per_share],
        marker_color=bar_colors,
        text=[f"€{current_price:.0f}", f"€{intrinsic_value_per_share:.0f}"],
        textposition="outside",
        textfont=dict(size=14),
        hovertemplate="<b>%{x}</b><br>€%{y:.2f}<extra></extra>",
    ),
    row=2,
    col=1,
)

# -----------------------------------------------------------------------------
# Panel (d): Valuation Indicator
# -----------------------------------------------------------------------------
fig_dcf.add_trace(
    go.Indicator(
        mode="number+delta",
        value=intrinsic_value_per_share,
        number={"prefix": "€", "font": {"size": 48}},
        delta={
            "reference": current_price,
            "relative": True,
            "valueformat": ".1%",
            "font": {"size": 20},
        },
        title={
            "text": "DCF Intrinsic Value<br><span style='font-size:12px;color:gray'>vs Market Price</span>"
        },
        domain={"x": [0.55, 0.95], "y": [0.0, 0.4]},
    ),
    row=2,
    col=2,
)

fig_dcf.update_layout(
    height=700,
    title_text=f"Figure 4: DCF Valuation Results — {ticker}",
    title_font=dict(size=16),
    template="plotly_white",
    showlegend=False,
    margin=dict(t=100, b=60, l=60, r=60),
)

fig_dcf.update_yaxes(title_text="€ Billion", row=1, col=2, range=[0, 160])
fig_dcf.update_yaxes(title_text="€ per Share", row=2, col=1, range=[0, 160])

fig_dcf.show()

## 6. Sensitivity Analysis

DCF valuations are highly sensitive to key assumptions. We analyze the impact of varying:

1. **WACC** (5% – 8%): Reflects uncertainty in cost of capital estimation
2. **Terminal Growth Rate** (1% – 2.5%): Long-run growth assumptions

The sensitivity matrix shows intrinsic value per share across different scenarios:

$$\text{Value}_{i,j} = f(\text{WACC}_i, g_j)$$

### 6.1 WACC–Growth Sensitivity Matrix

In [88]:
# =============================================================================
# Sensitivity Analysis: WACC vs Terminal Growth Rate
# =============================================================================


# -----------------------------------------------------------------------------
# Define Sensitivity Ranges
# -----------------------------------------------------------------------------
wacc_range = [0.05, 0.055, 0.06, 0.065, 0.07, 0.075, 0.08]
growth_range = [0.010, 0.015, 0.020, 0.025]

# Get forecast FCFs and terminal FCF from previous calculations
forecast_fcfs = df_forecast["FCF"].values
terminal_fcf_base = df_forecast["FCF"].iloc[-1]
net_debt_val = dcf_params["net_debt"]
shares = info.get("sharesOutstanding", 600_000_000)


# -----------------------------------------------------------------------------
# Build Sensitivity Matrix
# -----------------------------------------------------------------------------
def calculate_dcf_value(wacc_val, growth_val, fcfs, terminal_fcf, net_debt, shares_out):
    """Calculate intrinsic value per share for given WACC and growth rate."""
    pv_fcfs = sum(fcf / (1 + wacc_val) ** (t + 1) for t, fcf in enumerate(fcfs))
    tv = terminal_fcf * (1 + growth_val) / (wacc_val - growth_val)
    pv_tv = tv / (1 + wacc_val) ** len(fcfs)
    ev = pv_fcfs + pv_tv
    equity_val = ev - net_debt
    return equity_val / shares_out


# Generate sensitivity matrix
sensitivity_matrix = pd.DataFrame(
    [
        [
            calculate_dcf_value(w, g, forecast_fcfs, terminal_fcf_base, net_debt_val, shares)
            for g in growth_range
        ]
        for w in wacc_range
    ],
    index=[f"{w:.1%}" for w in wacc_range],
    columns=[f"{g:.1%}" for g in growth_range],
)

# Store for later
dcf_params["sensitivity_matrix"] = sensitivity_matrix
dcf_params["calculate_dcf_value"] = calculate_dcf_value

In [89]:
# =============================================================================
# Figure 5: Sensitivity Analysis Visualization
# =============================================================================

fig_sensitivity = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=(
        "(a) WACC–Growth Sensitivity Matrix",
        "(b) Key Driver Impact (Tornado)",
    ),
    specs=[[{"type": "heatmap"}, {"type": "bar"}]],
    horizontal_spacing=0.18,
)

# -----------------------------------------------------------------------------
# Panel (a): Heatmap
# -----------------------------------------------------------------------------
z_values = sensitivity_matrix.values
x_labels = list(sensitivity_matrix.columns)
y_labels = list(sensitivity_matrix.index)

# Find base case position
base_wacc_str = f"{wacc:.1%}"
base_growth_str = f"{terminal_growth_rate:.1%}"
base_wacc_idx = y_labels.index(base_wacc_str) if base_wacc_str in y_labels else 2
base_growth_idx = x_labels.index(base_growth_str) if base_growth_str in x_labels else 2

fig_sensitivity.add_trace(
    go.Heatmap(
        z=z_values,
        x=x_labels,
        y=y_labels,
        colorscale="RdYlGn",
        text=[[f"€{v:.0f}" for v in row] for row in z_values],
        texttemplate="%{text}",
        textfont={"size": 11},
        hovertemplate="WACC: %{y}<br>Growth: %{x}<br>Value: €%{z:.0f}<extra></extra>",
        colorbar=dict(title="€/Share", x=0.46, len=0.9),
    ),
    row=1,
    col=1,
)

# Mark base case with star
fig_sensitivity.add_trace(
    go.Scatter(
        x=[x_labels[base_growth_idx]],
        y=[y_labels[base_wacc_idx]],
        mode="markers+text",
        marker=dict(size=25, symbol="star", color="black", line=dict(color="white", width=2)),
        text=["★"],
        textfont=dict(size=20, color="gold"),
        textposition="middle center",
        name="Base Case",
        showlegend=False,
        hovertemplate=f"<b>Base Case</b><br>WACC: {base_wacc_str}<br>Growth: {base_growth_str}<br>Value: €{intrinsic_value_per_share:.0f}<extra></extra>",
    ),
    row=1,
    col=1,
)

# -----------------------------------------------------------------------------
# Panel (b): Tornado Chart
# -----------------------------------------------------------------------------
base_value = intrinsic_value_per_share

# WACC sensitivity
wacc_low = calculate_dcf_value(
    0.05, terminal_growth_rate, forecast_fcfs, terminal_fcf_base, net_debt_val, shares
)
wacc_high = calculate_dcf_value(
    0.08, terminal_growth_rate, forecast_fcfs, terminal_fcf_base, net_debt_val, shares
)

# Growth sensitivity
growth_low = calculate_dcf_value(wacc, 0.01, forecast_fcfs, terminal_fcf_base, net_debt_val, shares)
growth_high = calculate_dcf_value(
    wacc, 0.025, forecast_fcfs, terminal_fcf_base, net_debt_val, shares
)

# FCF sensitivity (±20%)
fcf_low = calculate_dcf_value(
    wacc, terminal_growth_rate, forecast_fcfs, terminal_fcf_base * 0.8, net_debt_val, shares
)
fcf_high = calculate_dcf_value(
    wacc, terminal_growth_rate, forecast_fcfs, terminal_fcf_base * 1.2, net_debt_val, shares
)

tornado_data = [
    {
        "var": "WACC<br>(5% → 8%)",
        "low": wacc_high,
        "high": wacc_low,
        "range": abs(wacc_low - wacc_high),
    },
    {
        "var": "Terminal Growth<br>(1% → 2.5%)",
        "low": growth_low,
        "high": growth_high,
        "range": abs(growth_high - growth_low),
    },
    {
        "var": "Terminal FCF<br>(±20%)",
        "low": fcf_low,
        "high": fcf_high,
        "range": abs(fcf_high - fcf_low),
    },
]
tornado_data.sort(key=lambda x: x["range"], reverse=True)

# Low side bars
fig_sensitivity.add_trace(
    go.Bar(
        y=[d["var"] for d in tornado_data],
        x=[d["low"] - base_value for d in tornado_data],
        orientation="h",
        name="Downside",
        marker_color="rgb(214, 39, 40)",
        text=[f"€{d['low']:.0f}" for d in tornado_data],
        textposition="outside",
        textfont=dict(size=10),
        hovertemplate="<b>%{y}</b><br>Value: €%{text}<extra></extra>",
    ),
    row=1,
    col=2,
)

# High side bars
fig_sensitivity.add_trace(
    go.Bar(
        y=[d["var"] for d in tornado_data],
        x=[d["high"] - base_value for d in tornado_data],
        orientation="h",
        name="Upside",
        marker_color="rgb(44, 160, 101)",
        text=[f"€{d['high']:.0f}" for d in tornado_data],
        textposition="outside",
        textfont=dict(size=10),
        hovertemplate="<b>%{y}</b><br>Value: €%{text}<extra></extra>",
    ),
    row=1,
    col=2,
)

# Base case line
fig_sensitivity.add_vline(x=0, line_dash="dash", line_color="black", line_width=2, row=1, col=2)
fig_sensitivity.add_annotation(
    x=0,
    y=1.15,
    text=f"Base: €{base_value:.0f}",
    showarrow=False,
    font=dict(size=11, color="black"),
    xref="x2",
    yref="paper",
)

fig_sensitivity.update_layout(
    height=500,
    title_text=f"Figure 5: Sensitivity Analysis — {ticker}",
    title_font=dict(size=16),
    template="plotly_white",
    barmode="overlay",
    legend=dict(orientation="h", yanchor="bottom", y=-0.25, xanchor="center", x=0.75),
    margin=dict(t=80, b=80, l=60, r=80),
)

fig_sensitivity.update_xaxes(title_text="Terminal Growth Rate", row=1, col=1)
fig_sensitivity.update_yaxes(title_text="WACC", row=1, col=1)
fig_sensitivity.update_xaxes(
    title_text="Δ from Base Case (€/share)", row=1, col=2, zeroline=True, range=[-80, 80]
)

fig_sensitivity.show()

### 6.2 Scenario Analysis

We define three scenarios reflecting different assumptions about BMW's EV transition:

#### Assumed Values by Scenario

| Scenario | WACC | Terminal Growth | Terminal Margin | Rationale |
|----------|------|-----------------|-----------------|-----------|
| **Bull** | **6.5%** | **2.5%** | **11.5%** | Successful EV transition; lower risk premium; margin expansion |

| **Base** | (computed) | **2.0%** | **9%** | Current conservative assumptions || **Bear** | **8.5%** | **1.0%** | **6.5%** | Competitive pressure; higher risk; margin compression |

In [90]:
# =============================================================================
# Scenario Analysis: Bull / Base / Bear Cases
# =============================================================================


def run_full_dcf_scenario(
    base_rev,
    rev_growth_rates,
    margin_trajectory,
    capex_pcts,
    da_pcts,
    tax_r,
    wacc_val,
    term_growth,
    net_debt_val,
    shares_out,
):
    """Run a complete DCF model with given assumptions."""
    fcfs = []
    prev_rev = base_rev
    for i in range(5):
        rev = prev_rev * (1 + rev_growth_rates[i])
        delta_rev = rev - prev_rev
        ebit = rev * margin_trajectory[i]
        nopat_val = ebit * (1 - tax_r)
        da_val = rev * da_pcts[i]
        capex_val = rev * capex_pcts[i]
        delta_nwc_val = delta_rev * 0.05
        fcf_val = nopat_val + da_val - capex_val - delta_nwc_val
        fcfs.append(fcf_val)
        prev_rev = rev
    term_fcf = fcfs[-1]
    return calculate_dcf_value(wacc_val, term_growth, fcfs, term_fcf, net_debt_val, shares_out)


# Define scenarios with realistic auto industry margins
# Auto industry EBIT margins: BMW historically 8-12%, industry avg 5-8%
# Bull: margin expansion to 10-11% (achievable with EV success)
# Base: stable 8-9% (current realistic level)
# Bear: margin compression to 6-7% (competitive pressure, still positive)
scenarios = {
    "Bull": {
        "wacc": 0.065,  # Lower WACC reflects lower risk premium
        "growth": 0.025,
        "rev_growth": [0.02, 0.03, 0.03, 0.03, 0.03],
        "margins": [0.095, 0.10, 0.105, 0.11, 0.115],  # Realistic bull: 9.5% → 11.5%
        "capex": [0.065, 0.06, 0.055, 0.055, 0.05],
        "color": "rgb(44, 160, 101)",
    },
    "Base": {
        "wacc": wacc,
        "growth": 0.02,
        "rev_growth": revenue_growth,
        "margins": ebit_margin_forecast,  # 8% → 9%
        "capex": capex_pct,
        "color": "rgb(55, 83, 109)",
    },
    "Bear": {
        "wacc": 0.085,  # Higher WACC reflects risk
        "growth": 0.01,
        "rev_growth": [0.0, 0.01, 0.01, 0.01, 0.01],
        "margins": [0.065, 0.06, 0.06, 0.065, 0.065],  # Margin compression: 6-6.5%
        "capex": [0.07, 0.065, 0.065, 0.06, 0.06],
        "color": "rgb(214, 39, 40)",
    },
}

# Calculate values
for _name, params in scenarios.items():
    params["value"] = run_full_dcf_scenario(
        base_revenue,
        params["rev_growth"],
        params["margins"],
        params["capex"],
        da_pct,
        tax_rate,
        params["wacc"],
        params["growth"],
        net_debt_val,
        shares,
    )

# Probability-weighted expected value
prob_weights = {"Bull": 0.25, "Base": 0.50, "Bear": 0.25}
expected_value = sum(scenarios[s]["value"] * prob_weights[s] for s in scenarios)

dcf_params["scenarios"] = scenarios
dcf_params["expected_value"] = expected_value

In [91]:
# =============================================================================
# Figure 6: Scenario Analysis & Final Valuation Summary
# =============================================================================

fig_final = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "(a) Scenario Valuation Range",
        "(b) Probability Distribution",
        "(c) Key Assumptions by Scenario",
        "(d) Investment Summary",
    ),
    specs=[
        [{"type": "bar"}, {"type": "bar"}],
        [{"type": "table"}, {"type": "indicator"}],
    ],
    vertical_spacing=0.18,
    horizontal_spacing=0.12,
)

# -----------------------------------------------------------------------------
# Panel (a): Scenario Bar Chart
# -----------------------------------------------------------------------------
scenario_names = list(scenarios.keys())
scenario_values = [scenarios[s]["value"] for s in scenario_names]
scenario_colors = [scenarios[s]["color"] for s in scenario_names]

fig_final.add_trace(
    go.Bar(
        x=scenario_names,
        y=scenario_values,
        marker_color=scenario_colors,
        text=[f"€{v:,.0f}" for v in scenario_values],
        textposition="inside",
        textfont=dict(size=14, color="white"),
        hovertemplate="<b>%{x} Case</b><br>Value: €%{y:,.0f}<extra></extra>",
    ),
    row=1,
    col=1,
)

# Market price reference
fig_final.add_hline(
    y=current_price,
    line_dash="dash",
    line_color="black",
    line_width=2,
    annotation_text=f"Market: €{current_price:.0f}",
    annotation_position="right",
    row=1,
    col=1,
)

# Expected value
fig_final.add_hline(
    y=expected_value,
    line_dash="dot",
    line_color="purple",
    line_width=2,
    annotation_text=f"Expected: €{expected_value:.0f}",
    annotation_position="left",
    row=1,
    col=1,
)

# -----------------------------------------------------------------------------
# Panel (b): Probability Distribution Visualization
# -----------------------------------------------------------------------------
fig_final.add_trace(
    go.Bar(
        x=scenario_names,
        y=[prob_weights[s] * 100 for s in scenario_names],
        marker_color=scenario_colors,
        text=[f"{prob_weights[s]:.0%}" for s in scenario_names],
        textposition="outside",
        textfont=dict(size=12),
        hovertemplate="<b>%{x}</b><br>Probability: %{y:.0f}%<extra></extra>",
    ),
    row=1,
    col=2,
)

# -----------------------------------------------------------------------------
# Panel (c): Assumptions Table
# -----------------------------------------------------------------------------
table_data = go.Table(
    header=dict(
        values=["<b>Parameter</b>", "<b>Bull</b>", "<b>Base</b>", "<b>Bear</b>"],
        fill_color="rgb(55, 83, 109)",
        font=dict(color="white", size=11),
        align="center",
    ),
    cells=dict(
        values=[
            ["WACC", "Terminal Growth", "Terminal Margin", "Intrinsic Value", "vs Market"],
            [
                f"{scenarios['Bull']['wacc']:.1%}",
                f"{scenarios['Bull']['growth']:.1%}",
                f"{scenarios['Bull']['margins'][-1]:.0%}",
                f"€{scenarios['Bull']['value']:,.0f}",
                f"{(scenarios['Bull']['value'] / current_price - 1):+.0%}",
            ],
            [
                f"{scenarios['Base']['wacc']:.1%}",
                f"{scenarios['Base']['growth']:.1%}",
                f"{scenarios['Base']['margins'][-1]:.0%}",
                f"€{scenarios['Base']['value']:,.0f}",
                f"{(scenarios['Base']['value'] / current_price - 1):+.0%}",
            ],
            [
                f"{scenarios['Bear']['wacc']:.1%}",
                f"{scenarios['Bear']['growth']:.1%}",
                f"{scenarios['Bear']['margins'][-1]:.0%}",
                f"€{scenarios['Bear']['value']:,.0f}",
                f"{(scenarios['Bear']['value'] / current_price - 1):+.0%}",
            ],
        ],
        fill_color=[["white", "rgb(232, 245, 233)", "rgb(227, 242, 253)", "rgb(255, 235, 238)"]]
        * 5,
        font=dict(size=11),
        align="center",
        height=25,
    ),
    domain={"x": [0.0, 0.48], "y": [0.0, 0.38]},
)
fig_final.add_trace(table_data, row=2, col=1)

# -----------------------------------------------------------------------------
# Panel (d): Final Verdict Indicator
# -----------------------------------------------------------------------------
verdict_color = (
    "green" if premium_discount < -0.1 else ("red" if premium_discount > 0.1 else "orange")
)
verdict_text = (
    "UNDERVALUED"
    if premium_discount < -0.1
    else ("OVERVALUED" if premium_discount > 0.1 else "FAIR VALUE")
)

fig_final.add_trace(
    go.Indicator(
        mode="number+delta",
        value=expected_value,
        number={"prefix": "€", "font": {"size": 40}},
        delta={
            "reference": current_price,
            "relative": True,
            "valueformat": ".1%",
            "font": {"size": 18},
        },
        title={
            "text": f"<b>Expected Value</b><br><span style='font-size:16px;color:{verdict_color}'>{verdict_text}</span>"
        },
        domain={"x": [0.55, 0.95], "y": [0.0, 0.38]},
    ),
    row=2,
    col=2,
)

fig_final.update_layout(
    height=750,
    title_text=f"Figure 6: Investment Analysis Summary — {ticker}",
    title_font=dict(size=16),
    template="plotly_white",
    showlegend=False,
    margin=dict(t=100, b=60, l=60, r=60),
)

fig_final.update_yaxes(title_text="Intrinsic Value (€)", row=1, col=1, range=[0, 200])
fig_final.update_yaxes(title_text="Probability (%)", row=1, col=2, range=[0, 70])

fig_final.show()

## 7. Conclusion

### 7.1 Investment Verdict

Based on our DCF analysis, we summarize the key findings:

| Criterion | Assessment |
|-----------|------------|
| **Methodology** | Two-stage DCF with 5-year explicit forecast + terminal value |
| **Base Case Value** | See valuation summary above |
| **Key Drivers** | WACC, terminal growth, EBIT margin trajectory |
| **Sensitivity** | High - small changes in WACC/growth significantly impact value |

### 7.2 Risk Factors

| Risk | Impact | Mitigation |
|------|--------|------------|
| EV transition costs | FCF pressure | Monitor CapEx trends |
| Margin compression | Lower terminal value | Track quarterly margins |
| Macro slowdown | Higher WACC, lower growth | Diversified geographic exposure |
| Competition (Tesla, Chinese OEMs) | Market share loss | Brand strength, Neue Klasse platform |

### 7.3 Monitoring Metrics

1. **EBIT Margin** — Target >16% by 2026 for thesis confirmation
2. **FCF** — Must return to positive territory in 2025
3. **ROIC vs WACC** — Spread must widen for value creation
4. **EV Unit Sales** — Neue Klasse adoption trajectory

In [92]:
# =============================================================================
# Figure 7: Executive Dashboard — Full DCF Model Summary
# =============================================================================

fig_dashboard = make_subplots(
    rows=2,
    cols=3,
    subplot_titles=(
        "Risk-Free Rate",
        "Cost of Equity",
        "WACC",
        "Intrinsic Value",
        "Upside Potential",
        "Model Confidence",
    ),
    specs=[
        [{"type": "indicator"}] * 3,
        [{"type": "indicator"}] * 3,
    ],
    vertical_spacing=0.35,
    horizontal_spacing=0.12,
)

# Row 1: Capital Cost Metrics
fig_dashboard.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=rf * 100,  # rf is the risk-free rate variable
        number={"suffix": "%", "font": {"size": 28}},
        gauge={
            "axis": {"range": [0, 6], "ticksuffix": "%"},
            "bar": {"color": "steelblue"},
            "steps": [{"range": [0, 3], "color": "lightgray"}, {"range": [3, 6], "color": "gray"}],
        },
        domain={"row": 0, "column": 0},
    ),
    row=1,
    col=1,
)

fig_dashboard.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=cost_of_equity * 100,  # cost_of_equity from CAPM
        number={"suffix": "%", "font": {"size": 28}},
        gauge={
            "axis": {"range": [0, 15], "ticksuffix": "%"},
            "bar": {"color": "darkorange"},
            "steps": [{"range": [0, 8], "color": "lightgray"}, {"range": [8, 15], "color": "gray"}],
        },
        domain={"row": 0, "column": 1},
    ),
    row=1,
    col=2,
)

fig_dashboard.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=wacc * 100,
        number={"suffix": "%", "font": {"size": 28}},
        gauge={
            "axis": {"range": [0, 12], "ticksuffix": "%"},
            "bar": {"color": "darkgreen"},
            "steps": [{"range": [0, 6], "color": "lightgray"}, {"range": [6, 12], "color": "gray"}],
            "threshold": {"line": {"color": "red", "width": 3}, "value": 8},
        },
        domain={"row": 0, "column": 2},
    ),
    row=1,
    col=3,
)

# Row 2: Valuation Metrics
upside = (intrinsic_value_per_share / current_price - 1) * 100
upside_color = "green" if upside > 10 else ("red" if upside < -10 else "orange")

fig_dashboard.add_trace(
    go.Indicator(
        mode="number+delta",
        value=intrinsic_value_per_share,
        number={"prefix": "€", "font": {"size": 32}},
        delta={"reference": current_price, "relative": False, "position": "bottom"},
        domain={"row": 1, "column": 0},
    ),
    row=2,
    col=1,
)

fig_dashboard.add_trace(
    go.Indicator(
        mode="gauge+number+delta",
        value=upside,
        number={"suffix": "%", "font": {"size": 28, "color": upside_color}},
        delta={"reference": 0, "position": "bottom"},
        gauge={
            "axis": {"range": [-50, 100], "ticksuffix": "%"},
            "bar": {"color": upside_color},
            "steps": [
                {"range": [-50, -10], "color": "rgba(255,0,0,0.2)"},
                {"range": [-10, 10], "color": "rgba(255,165,0,0.2)"},
                {"range": [10, 100], "color": "rgba(0,128,0,0.2)"},
            ],
            "threshold": {"line": {"color": "black", "width": 2}, "value": 0},
        },
        domain={"row": 1, "column": 1},
    ),
    row=2,
    col=2,
)

# Confidence score based on sensitivity spread
sensitivity_values = sensitivity_matrix.values.flatten()
cv = np.std(sensitivity_values) / np.mean(sensitivity_values)  # coefficient of variation
confidence = max(0, min(100, (1 - cv) * 100))  # lower CV = higher confidence

fig_dashboard.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=confidence,
        number={"suffix": "%", "font": {"size": 28}},
        gauge={
            "axis": {"range": [0, 100], "ticksuffix": "%"},
            "bar": {"color": "purple"},
            "steps": [
                {"range": [0, 40], "color": "rgba(255,0,0,0.3)"},
                {"range": [40, 70], "color": "rgba(255,165,0,0.3)"},
                {"range": [70, 100], "color": "rgba(0,128,0,0.3)"},
            ],
        },
        domain={"row": 1, "column": 2},
    ),
    row=2,
    col=3,
)

fig_dashboard.update_layout(
    height=650,
    title_text=f"Figure 7: DCF Model Executive Dashboard — {ticker}",
    title_font=dict(size=16),
    template="plotly_white",
    margin=dict(t=100, b=60, l=50, r=50),
)

fig_dashboard.show()